## Example model: Sugar Factory

In [1]:
from model import Model
from chunk import Chunk
import numpy as np
import matplotlib.pyplot as plt
import random

m = Model()
m.rt = -5.0

First define the function that calculates the output of the factory

In [2]:
def sugar(w):
    new = 2 * w - production + random.randint(-1, 1)
    if new < 1:
        new = 1
    elif new > 12:
        new = 12
    return new

Set day to 1 and production to 5

In [3]:
day = 1
production = 5

The following function calculates makes a decision for one day at the factory. For the first 10 days it just picks a random number of workers in order to create some examples in memory. After that, it will use partial matching to retrieve an experience form memory that is most similar to the current situation. That is: it retrieves an experience with the current production as "yesterday", today's production at 9 (because that is the goal). It then uses the retrieved number of workers as its decision. The result is then added to declarative memory.

In [4]:
def do_a_day():
    global day
    global production
    print("Day %d" % day)
    if day < 10:
        workers = random.randint(1,10)
    else:
        request = Chunk(name = "request", slots = {"yesterday" : production, "today" : 9})
        result, latency = m.retrieve_partial(request, trace = True)
        if result == None:
            workers = random.randint(1,10)
        else:
            workers = result.slots["workers"]
            print("Retrieved chunk:")
            print(result)
    output = sugar(workers)
    print("New experience:")
    new_chunk = Chunk(name = "experience" + str(day), slots = {"yesterday" : production, "workers" : workers, "today": output})
    m.add_encounter(new_chunk)
    day += 1
    production = output
    m.time += 5.0
    print(new_chunk)

If we now run the model for a number of days, we can observe its decisions. Remember that the first 10 experiences are random.

In [5]:
for i in range(20):
    do_a_day()

Day 1
New experience:
Chunk experience1
Slots: {'yesterday': 5, 'workers': 3, 'today': 1}
Encounters: [0]
Fan: 0

Day 2
New experience:
Chunk experience2
Slots: {'yesterday': 1, 'workers': 1, 'today': 1}
Encounters: [5.0]
Fan: 0

Day 3
New experience:
Chunk experience3
Slots: {'yesterday': 1, 'workers': 6, 'today': 11}
Encounters: [10.0]
Fan: 0

Day 4
New experience:
Chunk experience4
Slots: {'yesterday': 11, 'workers': 7, 'today': 2}
Encounters: [15.0]
Fan: 0

Day 5
New experience:
Chunk experience5
Slots: {'yesterday': 2, 'workers': 8, 'today': 12}
Encounters: [20.0]
Fan: 0

Day 6
New experience:
Chunk experience6
Slots: {'yesterday': 12, 'workers': 4, 'today': 1}
Encounters: [25.0]
Fan: 0

Day 7
New experience:
Chunk experience7
Slots: {'yesterday': 1, 'workers': 5, 'today': 10}
Encounters: [30.0]
Fan: 0

Day 8
New experience:
Chunk experience8
Slots: {'yesterday': 10, 'workers': 8, 'today': 6}
Encounters: [35.0]
Fan: 0

Day 9
New experience:
Chunk experience9
Slots: {'yesterday': 6